# User Constraints Demo Notebook

In [ ]:
"""
 BSD 2-Clause License

 Copyright (c) 2024, AI4Society Research Group

 Redistribution and use in source and binary forms, with or without
 modification, are permitted provided that the following conditions are met:

 1. Redistributions of source code must retain the above copyright notice, this
    list of conditions and the following disclaimer.

 2. Redistributions in binary form must reproduce the above copyright notice,
    this list of conditions and the following disclaimer in the documentation
    and/or other materials provided with the distribution.

 THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS"
 AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE
 IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE
 DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE
 FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL
 DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR
 SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER
 CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY,
 OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE
 OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.
"""

## Import class and set user configuration
- Import class.
- Set and define the number of constraints.
- For each constraint, let user determine whether or not they want that specific item in their meal. Should they not provide a preference, the system assumes that they are neutral towards that preference.

Note: Do note that the code will raise warnings if the number of constraints specified do not match the number of constraints defined.

In [1]:
# Import the class
import sys
sys.path.insert(1, '../metrics/')
import user_constraints as constraints

# Initialize class
c=constraints.User_Constraints()

Class initiated


In [2]:
# Define user configuration with N constraints
c.set_num_constraints(3)

# Return how many constraints a user has specified
c.get_num_constraints()

3

In [3]:
# Define preferences for each constraint. -1: No, 0: Neutral, 1: Yes
c.define_constraints({'HasMeat': -1, 'HasNuts': -1})

# See all constraints
c.get_constraints()

{'HasDairy': 0, 'HasMeat': -1, 'HasNuts': -1}

## Add new constraints, remove any if needed.

In [4]:
# Add new constraint
c.add_new_constraint('HasFish', 1)

# See all constraints
c.get_constraints()

{'HasDairy': 0, 'HasMeat': -1, 'HasNuts': -1, 'HasFish': 1}

In [5]:
# Remove a constraint
c.remove_constraint('HasFish')

# See all constraints
c.get_constraints()

{'HasDairy': 0, 'HasMeat': -1, 'HasNuts': -1}

# Add food items with annotated food roles.
- A food item is annotated with any common warnings. E.g., 'HasNuts', 'HasMeat', etc.
- A food item that is NOT annotated with such warnings is implied to not have any. 

In [6]:
# Add a single annotated food item
c.add_annotated_food_item('omelet', ['HasEggs'])

# Add multiple annotated food items at once
c.add_multiple_annotated_food_items({'trail mix': ['HasNuts'], 
                                    'granola bar': ['HasNuts'], 
                                    'walnut cake': ['HasNuts'],
                                    'black beans taco': [], 
                                    'cheese pizza': ['HasDairy'], 
                                    'creamy salmon pasta': ['HasDairy', 'HasFish'],
                                    'chicken pasta': ['HasMeat'],
                                    'fruit tart': [],
                                    'tea':[]
                                   })

# See all food items
c.get_annotated_food_items()

{'omelet': ['HasEggs'],
 'trail mix': ['HasNuts'],
 'granola bar': ['HasNuts'],
 'walnut cake': ['HasNuts'],
 'black beans taco': [],
 'cheese pizza': ['HasDairy'],
 'creamy salmon pasta': ['HasDairy', 'HasFish'],
 'chicken pasta': ['HasMeat'],
 'fruit tart': [],
 'tea': []}

## Calculating user constraints score

A user can input their preferences to the system in the following format. For a food item 'X':
- -1: user does not want to be recommended X in their meal.
- 0: user is neutral towards X being recommended in their meal.
- 1: user is positive towards X being recommended in their meal. They prefer it, and depending on the type of constraint we are using, we may or may not penalize a meal for including / not including that item.

We have two types of constraints:
1. Hard constraints - food items that a user states that he explicitly states that he wants. Denoted as -1 or 1.
2. Soft constraints - food items that a user states having a preference to. Denoted as 1.

If the `hard_constraints` flag is enabled, then we penalize a meal if it does not include the constraints user has provided. E.g., for a constraint set such as {'HasDairy': 0, 'HasMeat': -1, 'HasNuts': 1}:
- Regardless of whether a recommendation includes or does not include dairy items, we do not take any action.
- If even one of the items in a recommendation contains meat, we penalize the recommendation.
- If at least one of the items in a recommendation does NOT contain nuts, we penalize the recommendation.

If the `hard_constraints` flag is disabled, then we penalize a meal only if it does not satisfy the negative constraints user has provided. E.g., for a constraint set such as {'HasDairy': 0, 'HasMeat': -1, 'HasNuts': 1}:
- Regardless of whether a recommendation includes or does not include dairy items, we do not take any action.
- Regardless of whether a recommendation includes or does not include nut items, we do not take any action.
- But if even one of the items in a recommendation contains meat, then we penalize the recommendation.

In [7]:
# Test case 1: a meal rec is empty
recommendation={}

c.calc_config(recommendation)

c.get_config()

1.0

In [8]:
# Test case 2: a meal recommendation has all preferences correctly adhered to
c.define_constraints({'HasMeat': -1, 
                      'HasNuts': -1, 
                      'HasDairy': 0})

recommendation={'Beverage':'tea', 
                'Main Course': 'omelet', 
                'Side Dish': 'black beans taco', 
                'Dessert':'fruit'}

c.calc_config(recommendation)

c.get_config()

1.0

In [9]:
# Test case 3: a meal recommendation violates a constraint
recommendation={'Beverage':'tea', 
                'Main Course': 'omelet', 
                'Side Dish': 'black beans taco', 
                'Dessert':'walnut cake'}   # <--- violated constraint

c.calc_config(recommendation)

c.get_config()

0.6666666666666667

In [10]:
# Test case 4: a meal recommendation violates all constraints
c.define_constraints({'HasMeat': -1, 
                      'HasNuts': -1, 
                      'HasDairy': -1})

recommendation={'Main Course': 'cheese pizza', 
                'Side Dish': 'chicken pasta', 
                'Dessert':'walnut cake'}   

c.calc_config(recommendation)

c.get_config()

0.0

In [11]:
# Test case 5: a meal recommendation satisfies all constraints
c.define_constraints({'HasMeat': 1, 
                      'HasNuts': 1,  
                      'HasDairy': 1})

recommendation={'Main Course': 'omelet', 
                'Side Dish': 'black bean taco', 
                'Dessert':'fruit tart'}   

c.calc_config(recommendation)

c.get_config()

1.0

In [12]:
# Test case 6: same as test case #5 but the hard_constraints flag is ENABLED and meal rec violates all constraints 
c.define_constraints({'HasMeat': 1, 
                      'HasNuts': 1,  
                      'HasDairy': 1})

recommendation={'Main Course': 'omelet', 
                'Side Dish': 'black bean taco', 
                'Dessert':'fruit tart'}   

c.calc_config(recommendation, hard_constraints=True)

c.get_config()

0.0